In [1]:
import string 
import numpy as np
import pandas as pd


In [2]:
##reading the contents of txt file into a list
with open("imdb_labelled.txt") as f:
    conntent=f.readlines()

In [3]:
conntent[:5] 

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  \t0\n',
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  \t0\n',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  \t0\n',
 'Very little music or anything to speak of.  \t0\n',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  \t1\n']

In [4]:
##Removing the newline character from the list using .strip()
newcon=[]
for line in conntent:
    line=line.strip()## removing newline character
    newcon.append(line)
newcon[:6]

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  \t0',
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  \t0',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  \t0',
 'Very little music or anything to speak of.  \t0',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  \t1',
 "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty.  \t0"]

In [5]:
"""splitting the sentiment values(0,1) from the sentence in list and 
            stroing it in seperate lists"""
review=[]
for x in newcon:
    x=x.split("\t")[0]
    review.append(x)##splitting review and adding to list
review[:5]

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  ',
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  ',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  ',
 'Very little music or anything to speak of.  ',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  ']

In [6]:
label=[]
for x in newcon:
    x=x.split("\t")[1]
    label.append(x)## splitting labels and adding to list
label[:6]

['0', '0', '0', '0', '1', '0']

In [7]:
##Creating a dictionary to create a dataframe for pre-processing
connt={"review":review,"label":label}
df=pd.DataFrame(connt) ## creating data frame
print(df.shape)
df.head(10)

(1000, 2)


,review,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
5,"The rest of the movie lacks art, charm, meanin...",0
6,Wasted two hours.,0
7,Saw the movie today and thought it was a good ...,1
8,A bit predictable.,0
9,Loved the casting of Jimmy Buffet as the scien...,1


In [8]:

#removing stopwords from each row of reviews using anonymous function
import nltk
from nltk.corpus import stopwords ##stopword removal
stop = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(stop[:10])
df.head(10)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


,review,label
0,"A very, very, slow-moving, aimless movie distr...",0
1,"Not sure lost - flat characters audience, near...",0
2,Attempting artiness black & white clever camer...,0
3,Very little music anything speak of.,0
4,The best scene movie Gerardo trying find song ...,1
5,"The rest movie lacks art, charm, meaning... If...",0
6,Wasted two hours.,0
7,"Saw movie today thought good effort, good mess...",1
8,A bit predictable.,0
9,Loved casting Jimmy Buffet science teacher.,1


In [9]:
df['review'] = df['review'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))##punctuation removal
print(string.punctuation)
df.head(10)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


,review,label
0,A very very slowmoving aimless movie distresse...,0
1,Not sure lost flat characters audience nearly...,0
2,Attempting artiness black white clever camera...,0
3,Very little music anything speak of,0
4,The best scene movie Gerardo trying find song ...,1
5,The rest movie lacks art charm meaning If empt...,0
6,Wasted two hours,0
7,Saw movie today thought good effort good messa...,1
8,A bit predictable,0
9,Loved casting Jimmy Buffet science teacher,1


In [10]:
df["review"] = df["review"].apply(lambda x: x.lower())    ##converting to lower case
df.head(10)

,review,label
0,a very very slowmoving aimless movie distresse...,0
1,not sure lost flat characters audience nearly...,0
2,attempting artiness black white clever camera...,0
3,very little music anything speak of,0
4,the best scene movie gerardo trying find song ...,1
5,the rest movie lacks art charm meaning if empt...,0
6,wasted two hours,0
7,saw movie today thought good effort good messa...,1
8,a bit predictable,0
9,loved casting jimmy buffet science teacher,1


<h3> Deep NN training</h3>

In [11]:
from keras.models import Sequential ##Sequential layer
from keras.layers import Dense
from keras.layers import LSTM   
from keras.layers import SpatialDropout1D
from keras.layers.embeddings import Embedding 
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [12]:
preview=df["review"].tolist()

In [13]:
max_review_length = 200
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preview)

Xt = tokenizer.texts_to_sequences(preview)##tokenize
Xt = sequence.pad_sequences(Xt, maxlen= max_review_length) ##padding data for NN
print('Shape :', Xt.shape)


Shape : (1000, 200)


In [14]:
y=df["label"].tolist()
Y=pd.get_dummies(y).values 

Y.shape

(1000, 2)

In [15]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xt,Y,test_size=0.2) ##80% training data
xtrain[:1]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [16]:
EMBEDDING_DIM = 200
model = Sequential()
model.add(Embedding(10000, EMBEDDING_DIM, input_length=Xt.shape[1])) ## input layer using 200 nodes
model.add(SpatialDropout1D(0.2))
model.add(LSTM(250, dropout=0.2,return_sequences=True))  ##
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))  ##
model.add(Dense(2, activation='softmax'))   ## output layer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 200)          2000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 200, 200)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 250)          451000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               140400    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 2,591,602
Trainable params: 2,591,602
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
epochs = 10 
batch_size = 80
model.fit(xtrain,ytrain,
          epochs=epochs, 
          batch_size=batch_size)   ##training 

C:\Users\Yashwanth.S\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
800/800 [==============================] - 75s 94ms/step - loss: 0.6930 - accuracy: 0.5100
Epoch 2/10
800/800 [==============================] - 61s 76ms/step - loss: 0.6786 - accuracy: 0.5663
Epoch 3/10
800/800 [==============================] - 53s 66ms/step - loss: 0.6101 - accuracy: 0.6837
Epoch 4/10
800/800 [==============================] - 48s 60ms/step - loss: 0.3990 - accuracy: 0.8288
Epoch 5/10
800/800 [==============================] - 59s 74ms/step - loss: 0.1865 - accuracy: 0.9300
Epoch 6/10
800/800 [==============================] - 43s 53ms/step - loss: 0.1149 - accuracy: 0.9538
Epoch 7/10
800/800 [==============================] - 48s 60ms/step - loss: 0.0700 - accuracy: 0.9750
Epoch 8/10
800/800 [==============================] - 48s 59ms/step - loss: 0.0344 - accuracy: 0.9962
Epoch 9/10
800/800 [==============================] - 43s 54ms/step - loss: 0.0221 - accuracy: 0.9925
Epoch 10/10
800/800 [==============================] - 45s 56ms/step - loss: 0.016

In [20]:
ypred=model.predict(xtest)

loss, acc = model.evaluate(xtest, ytest, verbose=2,
                            batch_size=12)
print("loss: "+str(loss))
print("Validation accuracy: "+str(acc))  

loss: 1.4073146461695432
Validation accuracy: 0.7250000238418579


In [21]:
new = ["very boring bad "]  ## use your own sentence
     
seq = tokenizer.texts_to_sequences(new)
padded = sequence.pad_sequences(seq, maxlen=max_review_length)
pred = model.predict(padded)
word1=np.argmax(pred)
print(pred)
if word1==0:
    print("negative ")
else:
    print("positive")

[[9.9967754e-01 3.2246695e-04]]
negative 


In [64]:
from hpsklearn import HyperoptEstimator
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing
from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


C:\Users\Yashwanth.S\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [65]:
modeler = HyperoptEstimator( classifier=any_classifier('clf'),  
                            algo=tpe.suggest, trial_timeout=300)

In [73]:
trainy=[]
for i in range (len(ytrain)):
  trainy.append(ytrain[i][0])  
trainy

[0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,


In [71]:
modeler.fit(xtrain,trainy)

100%|████████████████████████████████████████████████████████████| 10/10 [00:11<00:00, 11.79s/trial, best loss: 0.4625]


In [72]:
print(modeler.best_model())

{'learner': GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.022835300313273254, loss='deviance',
              max_depth=3, max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=4, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=119,
              presort='auto', random_state=3, subsample=1.0, verbose=0,
              warm_start=False), 'preprocs': (), 'ex_preprocs': ()}


In [74]:
ypred1=modeler.predict(xtest)
ypred1

In [88]:
z = pd.get_dummies(ypred1).values 

In [89]:
from sklearn.metrics import accuracy_score
ypred1=modeler.predict(xtest)

accuracyScore1 = accuracy_score(ytest,z)*100

print("Prediction Accuracy :",accuracyScore1) 

Prediction Accuracy : 41.5
